### Help: https://youtu.be/1gQR24B3ISE

In [1]:
import torch
import torchvision

import pandas as pd

from tqdm import tqdm

import cv2 as cv
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split

df_28x28 = pd.read_csv('numbers.csv')
df_38x28 = pd.read_csv('digits_only_numbers.csv')

df = pd.concat([df_28x28, df_38x28], ignore_index=True)

max_val = 33
max_label_sample = 350

# fix digits
df.numbers.loc[df.numbers == '00'] = 0
for _ in range(160):
    if _ <= 152:
        df.numbers.loc[df.numbers == f'{_}'] = _
        # relabel: partially blurry > blurry
        for e in [f'b{_}', f'{_}b']:
            df.numbers.loc[df.numbers == e] = 'b'
        # relabel icon issue numbers as numbers
        for e in [f'i{_}']:
            df.numbers.loc[df.numbers == e] = _
    else:
        df = df.loc[df.numbers != f'{_}']

# fix nulls (standardize)
df.numbers.loc[df.numbers == 'b'] = ''
df.numbers.loc[df.numbers == 'e'] = ''
df.numbers.loc[df.numbers == 'r'] = ''
df.numbers.loc[df.numbers == 'n'] = ''
df.numbers.loc[df.numbers == ''] = 153

# 0-9 only
df = df.loc[df.numbers != '']
df = df.loc[df.numbers <= max_val]
for value in df.numbers.unique():
    c = len(df.loc[df.numbers==value])
    if c > max_label_sample:
        temp_df = df.loc[df.numbers == value].sample(max_label_sample)
        df = df.loc[df.numbers != value]
        df = pd.concat([df, temp_df])
    print(f'{value} | {len(df.loc[df.numbers==value])}')


file_paths = df.file_path.values
# load & resize image arrays
image_arrays = []
for path in file_paths:
    base_size = Image.open(path).size
    if base_size == (38, 28):
        img = cv.imread(path, cv.IMREAD_GRAYSCALE)
        img = Image.fromarray(img)
        img = img.crop((0, 0-5, 38, 28+5))
        img = np.array(img)
        img = cv.resize(img, (50, 50))
        image_arrays.append(img)
    elif base_size == (28, 28):
        img = cv.imread(path, cv.IMREAD_GRAYSCALE)
        img = Image.fromarray(img)
        img = img.crop((0-3, 0-5, 28+7, 28+5))
        img = np.array(img)
        img = cv.resize(img, (50, 50))
        image_arrays.append(img)
    else:
        raise Exception(f'\nerror: unknown size | {base_size}')

# image labels
y = df.numbers.values

# zip loaded images with their file paths
X = [[img, path] for img, path in zip(image_arrays, file_paths)]

# shuffle and split data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

# cut file paths from training & testing sets 
train_file_paths = [fp for img, fp in X_train]
test_file_paths = [fp for img, fp in X_test]

# cut image arrays from training & testing sets
X_train = [img for img, fp in X_train]
X_test = [img for img, fp in X_test]


train = [[img, np.eye(max_val+1)[label]] for img, label in zip(X_train, y_train)]
test = [[img, np.eye(max_val+1)[label]] for img, label in zip(X_test, y_test)]

np.save('training_data.npy', train)
np.save('testing_data.npy', test)

training_data = np.load('training_data.npy', allow_pickle=True)
len(training_data)

0 | 350
2 | 350
3 | 350
4 | 350
1 | 350
21 | 120
5 | 350
6 | 287
8 | 332
7 | 248
9 | 350
10 | 350
11 | 350
12 | 350
14 | 266
15 | 345
16 | 316
13 | 176
17 | 167
18 | 236
19 | 180
24 | 69
27 | 39
20 | 186
22 | 86
23 | 93
25 | 81
26 | 51
33 | 64
32 | 21
29 | 81
28 | 89
30 | 76
31 | 23


C:\Users\war23\Anaconda3\envs\open\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


6418

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, max_val+1) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=34, bias=True)
)


In [3]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

In [4]:
X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [5]:
VAL_PCT = 0.3  # lets reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)
print(val_size)

1925


In [6]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [7]:
print(len(train_X), len(test_X))

4493 1925


In [8]:
BATCH_SIZE = 80
EPOCHS = 4

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

        
    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|                                                                                           | 0/57 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.022386200726032257


  0%|                                                                                           | 0/57 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.010368489660322666


  0%|                                                                                           | 0/57 [00:00<?, ?it/s]

Epoch: 2. Loss: 0.010404382832348347


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [00:14<00:00,  3.88it/s]

Epoch: 3. Loss: 0.009877195581793785


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

100%|█████████████████████████████████████████████████████████████████████████████| 1925/1925 [00:04<00:00, 390.05it/s]

Accuracy:  0.925
